<div align="Center">

# Three-Link Robot Arm

## ENGR 635: Final Project

Andrew Albright  
andrew.albright1@louisiana.edu
</div>

In [1]:
# Import the SymPy Module
import sympy

# Import the necessary sub-modules and methods for dynamics
from sympy.physics.mechanics import dynamicsymbols
from sympy.physics.mechanics import inertia, LagrangesMethod, Lagrangian
from sympy.physics.mechanics import Particle, Point, ReferenceFrame, RigidBody

# initiate better printing of SymPy results
# sympy.init_printing()

# import NumPy with namespace np
import numpy as np

# import the scipy ODE solver
from scipy.integrate import odeint, solve_ivp

# import the plotting functions from matplotlib
import matplotlib.pyplot as plt

# set up the notebook to display the plots inline
%matplotlib inline

In [2]:
# Define the genearlized coordinate
theta1, theta2, theta3, Torq1, Torq2, Torq3 = dynamicsymbols('theta_1 theta_2 theta_3 tau_1 tau_2 tau_3')

# Also define the first derivative
theta1_dot, theta2_dot, theta3_dot= dynamicsymbols('theta_1 theta_2 theta_3', 1)

# Define the symbols for the other paramters
m1, m2, m3, l1, l2, l3, I1, I2, I3, t = sympy.symbols('m_1 m_2 m_3 l_1 l_2 l_3 I_1 I_2 I_3 t')

In [3]:
# Define the Newtonian reference frame
N = ReferenceFrame('N')

# Define a body-fixed frames
A = N.orientnew('A', 'Axis', [theta1, N.z]) # Fixed to link 1
B = A.orientnew('B', 'Axis', [theta2, A.z]) # Fixed to link 2
C = B.orientnew('C', 'Axis', [theta3, B.z]) # Fixed to link 3

# Define an origin and set its velocity to 0
O = Point('O')
O.set_vel(N, 0 * N.x)

# Define the COM of the first link location and velocity
G1 = O.locatenew('G1', l1/2 * A.x)
G1.v2pt_theory(O, N, A)

# Define the point at the end of the first link and set its velocity
P = O.locatenew('P', l1 * A.x)
P.v2pt_theory(O, N, A)

# Define the COM of the second link location and velocity
G2 = P.locatenew('G2', l2/2 * B.x)
G2.v2pt_theory(P, N, B)

# Define the point at the end of the second link and set its velocity
Q = P.locatenew('Q', l2 * B.x)
Q.v2pt_theory(P, N, B)

# Define the COM of the third link location and velocity
G3 = Q.locatenew('G3', l3/2 * C.x)
G3.v2pt_theory(Q, N, C)

# Create the first link rigid bogy
I_link1 = inertia(A, 0, 0, I1)
link1 = RigidBody('link1', G1, A, m1, (I_link1, G1))

# Create the second link rigid bogy
I_link2 = inertia(B, 0, 0, I2)
link2 = RigidBody('link2', G2, B, m2, (I_link2, G2))

# Create the third link rigid bogy
I_link3 = inertia(C, 0, 0, I3)
link3 = RigidBody('link3', G3, C, m3, (I_link3, G3))

# Now, set the potential energy of the links
# All are zero
link1.potential_energy = 0
link2.potential_energy = 0
link3.potential_energy = 0

# Set up the force list - each item follows the form:
#    (the location where the force is applied, its magnitude and direction)
forces = [(A, (Torq1-Torq2) * A.z), (B, (Torq2-Torq3) * B.z), (C, Torq3 * C.z)]

# # Form the Lagrangian
L =  Lagrangian(N, link1, link2, link3)

# # Print the Lagrangian as a check
L

I_1*Derivative(theta_1(t), t)**2/2 + I_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*Derivative(theta_1(t), t)/2 + I_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*Derivative(theta_2(t), t)/2 + I_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*Derivative(theta_1(t), t)/2 + I_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*Derivative(theta_2(t), t)/2 + I_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*Derivative(theta_3(t), t)/2 + l_1**2*m_1*Derivative(theta_1(t), t)**2/8 + m_2*(l_1**2*Derivative(theta_1(t), t)**2 + l_1*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_2(t))*Derivative(theta_1(t), t) + l_2**2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))**2/4)/2 + m_3*(l_1**2*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_2(t))*Derivative(theta_1(t), t) + l_1*l_3*

In [4]:
# This creates a LagrangesMethod class instance that will 
# allow us to form the equations of motion, etc
LM = LagrangesMethod(L, [theta1, theta2, theta3], forcelist=forces, frame=N)

In [5]:
# Form the equations fo motion
EqMotion = LM.form_lagranges_equations()

# Print the simplified version of the equations of motion
# sympy.simplify(EqMotion)

In [6]:
# This can be a very expensive call to make, if you don't need it

# Make the call to set up in state-space-ish form q_dot = f(q, t)
lrhs = LM.rhs()

# Output the result
print(lrhs)

Matrix([[Derivative(theta_1(t), t)], [Derivative(theta_2(t), t)], [Derivative(theta_3(t), t)], [(-m_2*(-l_1*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_2(t))*Derivative(theta_2(t), t) - l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t))/2 - m_3*(-2*l_1*l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_2(t))*Derivative(theta_2(t), t) - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_1*l_3*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t) + Derivative(theta_3(t), t))*(-sin(theta_2(t))*cos(theta_3(t))*Derivative(theta_2(t), t) - sin(theta_2(t))*cos(theta_3(t))*Derivative(theta_3(t), t) - sin(theta_3(t))*cos(theta_2(t))*Derivative(theta_2(t), t) - sin(theta_3(t))*cos(theta_2(t))*Derivative(theta_3(t), t)) + l_1*l_3*(-sin(theta_2(t))*cos(theta_3(t))*Derivative(theta_2(t), t) - sin(theta_2(t))*cos(theta_3(t))*Derivative(theta_3(t), t) - sin(theta_3(t))*cos(theta_2(t))*Derivative(the

## Simulation

In [ ]:
# define the forcing function
def torq(t):
    # set the maximum torque output
    torq_max = 100.0
    
    # return a bang-bang command
#     return torq_max * (t > 0.5) - 2 * torq_max * (t > 1.0) + torq_max * (t > 1.5)
    return np.random.uniform(-torq_max, torq_max)

def tau_1(t):
    return torq(t)

def tau_2(t):
    return torq(t)    

def tau_3(t):
    return torq(t)

In [ ]:
def eq_of_motion(t, w, p):
    """ Equations of motion for the two link system"""
    
    theta_1, theta_2, theta_3, theta_1_dot, theta_2_dot, theta_3_dot = w
    m_1, m_2, m_3, l_1, l_2, l_3, I_1, I_2, I_3 = p
    
    sys_ode = [theta_1_dot, 
                theta_2_dot, 
                theta_3_dot, 
                (-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot)/2 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3))/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)) + tau_3(t))/(I_3 + l_3**2*m_3/4 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))**2/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))) - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot)/2 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3))/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)) + tau_3(t))/(I_3 + l_3**2*m_3/4 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))**2/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))) - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)) + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2), 
                (-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot)/2 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3))/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)) + tau_3(t))/(I_3 + l_3**2*m_3/4 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))**2/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))) - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)), 
                (-m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot)/2 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3))/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))*(-l_1*l_2*m_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot/2 + l_1*l_2*m_2*np.sin(theta_2)*theta_1_dot*theta_2_dot/2 + m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_1_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3) - np.sin(theta_3)*np.cos(theta_2))*(theta_1_dot + theta_2_dot + theta_3_dot)*theta_1_dot)/2 - m_3*(-2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(-m_2*(-l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot)/2 - m_3*(-2*l_1*l_2*(theta_1_dot + theta_2_dot)*np.sin(theta_2)*theta_2_dot - 2*l_1*l_2*np.sin(theta_2)*theta_1_dot*theta_2_dot + l_1*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot) + l_1*l_3*(-np.sin(theta_2)*np.cos(theta_3)*theta_2_dot - np.sin(theta_2)*np.cos(theta_3)*theta_3_dot - np.sin(theta_3)*np.cos(theta_2)*theta_2_dot - np.sin(theta_3)*np.cos(theta_2)*theta_3_dot)*theta_1_dot - l_2*l_3*(theta_1_dot + theta_2_dot)*np.sin(theta_3)*theta_3_dot - l_2*l_3*(theta_1_dot + theta_2_dot + theta_3_dot)*np.sin(theta_3)*theta_3_dot)/2 + tau_1(t) + tau_2(t) + tau_3(t))/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) + tau_2(t) + tau_3(t))/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)) + tau_3(t))/(I_3 + l_3**2*m_3/4 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2) - (I_3 + m_3*(l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_3 + m_3*(l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)*(I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2))**2/(I_2 + I_3 + l_2**2*m_2/4 + m_3*(2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2 - (I_2 + I_3 + m_2*(l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1*l_2*np.cos(theta_2) + l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)**2/(I_1 + I_2 + I_3 + l_1**2*m_1/4 + m_2*(2*l_1**2 + 2*l_1*l_2*np.cos(theta_2) + l_2**2/2)/2 + m_3*(2*l_1**2 + 4*l_1*l_2*np.cos(theta_2) + 2*l_1*l_3*(-np.sin(theta_2)*np.sin(theta_3) + np.cos(theta_2)*np.cos(theta_3)) + 2*l_2**2 + 2*l_2*l_3*np.cos(theta_3) + l_3**2/2)/2)))]
    
    
    return sys_ode

In [ ]:
# Set up the initial conditions for the solver
theta1_init = 15.0 * np.pi/180  # Initial angle (rad)
theta1_dot_init = 0.0           # Initial angular velocity (rad/s)
theta2_init = 15.0 * np.pi/180  # Initial angle (rad)
theta2_dot_init = 0.0           # Initial angular velocity (rad/s)
theta3_init = 15.0 * np.pi/180  # Initial angle (rad)
theta3_dot_init = 0.0           # Initial angular velocity (rad/s)

# Pack the initial conditions into an array
x0 = [theta1_init, theta2_init, theta3_init, theta1_dot_init, theta2_dot_init, theta3_dot_init]

# Define the numerical values for all the system constants
# and pack them into a list for passing
m1 = 10.0
m2 = 10.0
m3 = 10.0
l1 = 2.0
l2 = 2.0 
l3 = 2.0
I1 = 1/12 * m1 * l1**2
I2 = 1/12 * m2 * l2**2
I3 = 1/12 * m3 * l3**2

p = [m1, m2, m3, l1, l2, l3, I1, I2, I3]

In [ ]:
total_time = 5.0
sim_time = np.linspace(0.0, total_time, 5001) # 0-10s with 1001 points in between

# ODE solver parameters
abserr = 1.0e-9
relerr = 1.0e-9
max_step = 0.01

# Call the ODE solver.
timeseries = np.zeros((len(sim_time), len(x0)))
tau = sim_time[1] - sim_time[0]
ii = 0
for t in sim_time:
    sim_time_inner = np.linspace(t, t+tau, 10)
    resp = odeint(eq_of_motion, x0, sim_time_inner, args=(p,), atol=abserr, rtol=relerr,  hmax=max_step, tfirst=True)
    timeseries[ii,:] = resp[-1]


# # Create the time samples for the output of the ODE solver
# tau = sim_time[1] - sim_time[0]
# # sim_time = np.linspace(0.0, tau, 500) # 0-10s with 1001 points in between

# timeseries = np.zeros((len(sim_time), len(x0)))
# # Call the ODE solver.
# ii = 0
# for t in sim_time:
#     resp = solve_ivp(eq_of_motion, [t, t+tau], x0, t_eval=[t, t+tau], args=(p,))
#     timeseries[ii,:] = resp.y[:,-1]
#     x0 = resp.y[:,-1]
#     ii += 1

In [ ]:
# Set the plot size - 3x2 aspect ratio is best
fig = plt.figure(figsize=(6, 4))
ax = plt.gca()
plt.subplots_adjust(bottom=0.17, left=0.17, top=0.96, right=0.96)

# Change the axis units to serif
plt.setp(ax.get_ymajorticklabels(), family='serif', fontsize=18)
plt.setp(ax.get_xmajorticklabels(), family='serif', fontsize=18)

# Remove top and right axes border
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Only show axes ticks on the bottom and left axes
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# Turn on the plot grid and set appropriate linestyle and color
ax.grid(True,linestyle=':', color='0.75')
ax.set_axisbelow(True)

# Define the X and Y axis labels
plt.xlabel('Time (s)', family='serif', fontsize=22, weight='bold', labelpad=5)
plt.ylabel('Angle (deg)', family='serif', fontsize=22, weight='bold', labelpad=10)

# Plot the data
plt.plot(sim_time, timeseries[:,0] * 180/np.pi, linewidth=2, linestyle='-', label = r'$\theta_1$')
plt.plot(sim_time, timeseries[:,1] * 180/np.pi, linewidth=2, linestyle='--', label = r'$\theta_2$')
plt.plot(sim_time, timeseries[:,2] * 180/np.pi, linewidth=2, linestyle='-.', label = r'$\theta_3$')

# uncomment below and set limits if needed
# plt.xlim(0, 5)
# plt.ylim(-1, 5)

# Create the legend, then fix the fontsize
leg = plt.legend(loc='upper right', ncol = 2, fancybox=True)
ltext  = leg.get_texts()
plt.setp(ltext, family='serif', fontsize=20)

# Adjust the page layout filling the page using the new tight_layout command
plt.tight_layout(pad=0.5)

# Uncomment to save the figure as a high-res pdf in the current folder
# It's saved at the original 6x4 size
# plt.savefig('TwoLinkArm_AngularResponse.pdf')

fig.set_size_inches(9, 6) # Resize the figure for better display in the notebook

In [ ]:
# Now, let's define the planar positions of the elbow joint and endpoint
# Then, we'll plot them on a planar representation of the workspace
elbow_1_x = l1 * np.cos(timeseries[:,0])
elbow_1_y = l1 * np.sin(timeseries[:,0])
elbow_2_x = elbow_1_x + l2 * np.cos(timeseries[:,0] + timeseries[:,1])
elbow_2_y = elbow_1_y + l2 * np.sin(timeseries[:,0] + timeseries[:,1])
end_x = elbow_2_x + l3 * np.cos(timeseries[:,0] + timeseries[:,1] + timeseries[:,2])
end_y = elbow_2_y + l3 * np.sin(timeseries[:,0] + timeseries[:,1] + timeseries[:,2])

In [ ]:
# Set the plot size - 3x2 aspect ratio is best
fig = plt.figure(figsize=(6, 4))
ax = plt.gca()
plt.subplots_adjust(bottom=0.17, left=0.17, top=0.96, right=0.96)

# Change the axis units to serif
plt.setp(ax.get_ymajorticklabels(), family='serif', fontsize=18)
plt.setp(ax.get_xmajorticklabels(), family='serif', fontsize=18)

# Remove top and right axes border
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Only show axes ticks on the bottom and left axes
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# Turn on the plot grid and set appropriate linestyle and color
ax.grid(True,linestyle=':', color='0.75')
ax.set_axisbelow(True)

# Define the X and Y axis labels
plt.xlabel('Horizontal Position (m)', family='serif', fontsize=22, weight='bold', labelpad=5)
plt.ylabel('Vertical Position (m)', family='serif', fontsize=22, weight='bold', labelpad=10)

# Plot the data
plt.plot(elbow_1_x, elbow_2_y, linewidth=2, linestyle='-', label = r'Elbow 1')
plt.plot(elbow_2_x, elbow_2_y, linewidth=2, linestyle='--', label = r'Elbow 2')
plt.plot(end_x, end_y, linewidth=2, linestyle='-.', label = r'Endpoint')
plt.plot(0,0, linestyle='', marker='o', markersize=5, label ='', zorder = 99)

# uncomment below and set limits if needed
# plt.xlim(-4, 4)
# plt.ylim(-1, 4+1/3)

# Create the legend, then fix the fontsize
leg = plt.legend(loc='upper right', ncol = 1, fancybox=True)
ltext  = leg.get_texts()
plt.setp(ltext, family='serif', fontsize=20)

# Adjust the page layout filling the page using the new tight_layout command
plt.tight_layout(pad=0.5)

# Uncomment to save the figure as a high-res pdf in the current folder
# It's saved at the original 6x4 size
# plt.savefig('TwoLinkArm_PlanarResponse.pdf')

fig.set_size_inches(9, 6) # Resize the figure for better display in the notebook